In [ ]:
!git clone https://github.com/jkaminsky2/CSE-151A-Project-.git

Cloning into 'CSE-151A-Project-'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 72 (delta 29), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (72/72), 491.21 KiB | 4.96 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [ ]:
#!pip install scikeras
#!pip install keras_tuner
import keras
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import RepeatedKFold, cross_validate
from scikeras.wrappers import KerasClassifier
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from sklearn.metrics import accuracy_score, confusion_matrix
from kerastuner.tuners import RandomSearch
import tensorflow as tf
import numpy as np
from sklearn.utils.class_weight import compute_class_weight


<ipython-input-2-52fbe02ef5fb>:15: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
num_df = pd.read_csv("CSE-151A-Project-/final_num_df_cse151a.csv")
X = num_df.drop(['Class_$','Class_$$', 'Class_$$$', 'Class_'], axis=1).astype(float)
y = num_df[['Class_$$', 'Class_$$$', 'Class_']].astype(float)
X = X[['closed', 'location_cluster', 'rating', 'relevant_cat', 'cat_cluster','sentiment']]
X['cat_cluster'] += 1
X['location_cluster'] = X['location_cluster'].fillna(-1)
X['location_cluster'] += 1
display(X.shape)
display(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    min_delta=0,
    patience=10,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

a= y_train['Class_$$$'] * 3 + y_train['Class_$$'] * 2 + y_train['Class_']
a -= 1
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(a), y=a)
class_weight = {0: class_weights[0], 1: class_weights[1], 2: class_weights[2]}



In [ ]:
def build_model(hp):
    input_layer = Input(shape=6)
    x = input_layer

    for i in range(4):
        units = hp.Int(f'units_{i}', min_value=8, max_value=18, step=4)
        activation = hp.Choice(f'activation_{i}', ['relu', 'sigmoid', 'tanh'])
        x = Dense(units=units, activation=activation)(x)
    output = Dense(units=3, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=output)

    lr = hp.Float("learning_rate", min_value=0.0001, max_value=0.3, sampling="log")
    epochs = hp.Int('epochs', min_value=50, max_value=200, step=50)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss='categorical_focal_crossentropy', metrics=['accuracy'], sample_weight_mode=class_weight)
    return model

tuner = RandomSearch(
    hypermodel=build_model,
    objective='accuracy',
    max_trials=3,
    executions_per_trial=2,
    directory='save_directory',
    project_name='hyperparams_optimization')

tuner.search(X_train, y_train, epochs=100)

best_hyperparameters = tuner.get_best_hyperparameters()[0]

best_model = tuner.hypermodel.build(best_hyperparameters)

best_model.fit(X_train, y_train, epochs=best_hyperparameters['epochs'])

y_pred_probabilities = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probabilities, axis=1)
y_true_expanded = np.argmax(y_test.values, axis=1)
test_accuracy = accuracy_score(y_true_expanded, y_pred)

print("Test Accuracy:", test_accuracy)
print("Optimized Hyperparameters\nNumber of Units in Each Hidden Layer:", [best_hyperparameters[f'units_{i}'] for i in range(4)])
print("Activation Function for Each Hidden Layer:", [best_hyperparameters[f'activation_{i}'] for i in range(4)])
print(F"Learning Rate: {best_hyperparameters['learning_rate']}\nEpochs: {best_hyperparameters['epochs']}")

from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_true_expanded, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Trial 3 Complete [00h 00m 39s]
accuracy: 0.6375176310539246

Best accuracy So Far: 0.6382228434085846
Total elapsed time: 00h 01m 23s
Epoch 1/150
23/23 [==============================] - 2s 2ms/step - loss: 0.1530 - accuracy: 0.0141 
Epoch 2/150
23/23 [==============================] - 0s 2ms/step - loss: 0.1471 - accuracy: 0.0141
Epoch 3/150
23/23 [==============================] - 0s 3ms/step - loss: 0.1423 - accuracy: 0.0169
Epoch 4/150
23/23 [==============================] - 0s 2ms/step - loss: 0.1382 - accuracy: 0.0212
Epoch 5/150
23/23 [==============================] - 0s 2ms/step - loss: 0.1347 - accuracy: 0.0226
Epoch 6/150
23/23 [==============================] - 0s 2ms/step - loss: 0.1319 - accuracy: 0.0282
Epoch 7/150
23/23 [==============================] - 0s 2ms/step - loss: 0.1295 - accuracy: 0.0324
Epoch 8/150
23/23 [==============================] - 0s 2ms/step - loss: 0.1273 - accuracy: 0.0423
Epoch 9/150
23/23 [==============================] - 0s 2ms/step - loss: 